In [51]:
import webbrowser
from pprint import pprint
from py2neo import Graph, NodeMatcher

# Esercizio 3.1 - NLP e Knowledge Graphs

In questo esercizio vogliamo rappresentare e visualizzare dei dati testuali in un KG ed eseguire alcune operazioni di _Information Retrieval_ su di esso.

Il corpus utilizzato è _smallNaukriJobsData_, un sottoinsieme contenente le prime 30 istanze del dataset _Naukri Jobs Data_ (<https://www.kaggle.com/datasets/kuchhbhi/latest-30k-jobs-data?resource=download>).

Accediamo a _Neo4j_ tramite la libreria _py2neo_, la quale opera da client permettendo di eseguire operazioni sul grafo direttamente con codice python.

In [61]:
# accediamo a Neo4j
graph = Graph('neo4j://localhost:7687', auth=()) # inserire credenziali neo4j 

# se ci sono già dei nodi li cancelliamo in modo da avere un grafo vuoto
print('Numero di nodi nel grafo: ', len(graph.nodes))
graph.delete_all()
print('Numero di nodi nel grafo: ', len(graph.nodes))

Numero di nodi nel grafo:  62
Numero di nodi nel grafo:  0


Costruiamo il grafo inserendo le istanze contenute nel file "_utils/small Naukri Jobs Data.csv_".
Viene creato:
- un nodo per la compagnia e le sue info
- un nodo per l'annuncio con tutti i suoi dati

Colleghiamo quindi i due nodi con una relazione di tipo "Compagnia -- OFFER --> Lavoro".

In [62]:
# carichiamo i nodi nel grafo

print('Numero di nodi nel grafo: ', len(graph.nodes))

# a partire dal file .csv importiamo 30 istanze di annunci di lavoro
graph.run('''
LOAD CSV WITH HEADERS FROM 'file:///smallNaukriJobsData.csv' AS line
CREATE(:Job_post {job_post: line.job_post, exp_required: line.exp_required, salary_offered: line.salary_offered, job_location: line.job_location, job_description: line.job_description, required_skills: line.required_skills, Posted_as_on_22_5_2022: line.Posted_as_on_22_5_2022})
CREATE(:Company {company: line.company, company_rating: line.company_rating, company_review: line.company_review})
MERGE (job:Job_post {job_post: line.job_post})
MERGE (company:Company {company: line.company})
CREATE (company)-[:OFFER]->(job)
''')

print('Numero di nodi nel grafo: ', len(graph.nodes))

Numero di nodi nel grafo:  0
Numero di nodi nel grafo:  60


Aggiungiamo altri due nodi (_Jobs_ e _Companies_) rappresentanti le possibili classi di appartenenza dei nodi inseriti in precedenza. Colleghiamo quindi ogni nodo alla propria classe con una relazione _IS-A_.

In [63]:
# creiamo i nodi
graph.run('''
CREATE(:Jobs {class: "job"})
CREATE(:Companies {class: "company"})
''')

# colleghiamo i nodi già esistenti alle classi di appartenenza

graph.run('''
MERGE (jobs: Jobs {class: "job"})
MERGE (job: Job_post)
CREATE (job)-[:ISA]->(jobs)
''')

graph.run('''
MERGE (companies: Companies {class: "company"})
MERGE (company: Company)
CREATE (company)-[:ISA]->(companies)
''')

print('Numero di nodi nel grafo: ', len(graph.nodes))

Numero di nodi nel grafo:  62


## Visualizzazione

Visualizziamo il grafo in forma interattiva.

In [64]:
# visualizziamo i dati accedendo a Neo4J
url = 'http://localhost:7474'
webbrowser.open(url, new=2) # new=2 opens a new tab

True

Visualizziamo i dati anche sul notebook sotto forma di DataFrame.

In [56]:
# stampiamo i nodi del grafo anche nel notebook
data = graph.run('''
MATCH (n)
RETURN n
''').to_data_frame()
print(len(data))
data.head()

62


,n
0,"{'exp_required': '0-3 Yrs', 'job_description':..."
1,"{'company_review': '(2907 Reviews)', 'company'..."
2,"{'exp_required': '5-15 Yrs', 'job_description'..."
3,"{'company_review': '(544 Reviews)', 'company':..."
4,"{'exp_required': '11-13 Yrs', 'job_description..."


## Information Retrieval

Eseguiamo alcune operazioni di ricerca sul grafo eseguendo delle query basate sull'utilizzo di una __parola chiave__.

In [57]:
node_matcher = NodeMatcher(graph) # oggetto per matchare i nodi del grafo con le proprietà indicate.

Effettuiamo una ricerca degli annunci che hanno come sede lavorativa una specifica città.

In [65]:
city = 'Mumbai' # input('Insert the name of the city: ')
match = node_matcher.match(job_location=city).all()
pprint(match)

[{'Posted_as_on_22_5_2022': '8 DAYS AGO',
  'exp_required': '3-5 Yrs',
  'job_description': 'Any Graduate OR Bachelors degree in Computer Science, '
                     'Information Technology, System Ad...',
  'job_location': 'Mumbai',
  'job_post': 'Executive/ Sr. Executive - Information Technology',
  'required_skills': 'Installation\n'
                     'Windows Administration\n'
                     'system administration\n'
                     'Active directory\n'
                     'troubleshooting\n'
                     'EPBX System\n'
                     'FTP\n'
                     'configuration',
  'salary_offered': 'Not disclosed'},
 {'Posted_as_on_22_5_2022': '30+ DAYS AGO',
  'exp_required': '2-6 Yrs',
  'job_description': 'KEY RESPONSIBILITIES Responsible to map business '
                     'requirements into Industry Standard Te...',
  'job_location': 'Mumbai',
  'job_post': 'Assistant Manager- Information Technology',
  'required_skills': 'IT Skills\n'
   

Effettuiamo una ricerca delle compagnie in base al nome specificato.

In [66]:
company = 'IBM' # input('Insert the name of the company: ')
match = node_matcher.match(company=company).all()
pprint(match)

[{'company': 'IBM',
  'company_rating': '4.2',
  'company_review': '(13315 Reviews)'}]


Effettuiamo una ricerca delle posizioni aperte in una data compagnia.

In [67]:
comp = 'Stefanini'
results = graph.run('''
MATCH (company:Company {company: $c})-[:OFFER]->(m) RETURN m
''', parameters={'c': comp})
pprint(results.data())

[{'m': {'Posted_as_on_22_5_2022': '4 DAYS AGO',
        'exp_required': '2-7 Yrs',
        'job_description': 'The resource will be expected to cross '
                           'skill\\train and support multiple projects Must '
                           'b...',
        'job_location': 'Noida',
        'job_post': 'Information Technology Trainee',
        'required_skills': 'POP\n'
                           'Training\n'
                           'Basic\n'
                           'Networking\n'
                           'Intern\n'
                           'IT infrastructure\n'
                           'Infrastructure\n'
                           'ITIL process',
        'salary_offered': 'Not disclosed'}},
 {'m': {'Posted_as_on_22_5_2022': '30+ DAYS AGO',
        'exp_required': '4-9 Yrs',
        'job_description': 'You must work well within a team environment and '
                           'enjoy working in the office with your ...',
        'job_location': 'Kollam/Quilo

Il grafo realizzato può essere ridotto o ingrandito in maniera estremamente semplice (basterebbe cambiare il file .csv con il dataset completo oppure inserire manualmente altri nodi), la __scalabilità__ è infatti uno dei vantaggi dei KG.
Questa forma di rappresentazione dei dati è molto utile per l'esecuzione di svariati task di NLP (come l'_Information Retrieval_ qui implementato).
La possibilità di visualizzare il grafo rende inoltre molto intuitivi i collegamenti fra i vari elementi.